In [1]:
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime, timedelta
from pandas import DataFrame

In [2]:
df_2020 = pd.read_csv('C:/Users/ching/Desktop/practice/movie/data/df_2020.csv')
#df_2020.tail()
#df_2020.info()

In [3]:
#今年發行之作品
condition = df_2020['上映日期'].str.contains('2020')
df_2020_0 = df_2020[condition]
df_2020_0

,Unnamed: 0,序號,國別地區,中文片名,上映日期,申請人,出品,上映院數,銷售票數,週票數變動率,銷售金額,累計銷售票數,累計銷售金額
0,0,1,中華民國,還有一些樹,2020-01-04,蜂鳥影像有限公司,蜂鳥影像有限公司,1,67,39.58,14725.0,115.0,25400.0
1,1,2,美國,兔嘲男孩,2020-01-03,博偉電影股份有限公司,"BUENA VISTA INTERNATIONAL, INC.",65,"25,771",46.25,6022947.0,50223.0,11724767.0
2,2,3,美國,驅魔直播,2020-01-03,海樂影業股份有限公司,"THE CLEANSING HOUR, LLC.",43,"5,679",9.40,1255702.0,10870.0,2401379.0
3,3,4,美國,別告訴她,2020-01-03,車庫娛樂股份有限公司,"A24 INTERNATIONAL, LLC",40,"4,629",44.39,1025995.0,7835.0,1740609.0
4,4,5,日本,《少女與戰車 總集篇4DX》-第63屆戰車道全國高中生大會,2020-01-03,蜜蜂工房創藝有限公司,Hakuhodo DY music &amp## pictures Inc.,6,"1,617",12.06,738945.0,3060.0,1397105.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3905,99,100,日本,鳴鳥不飛：烏雲密布,2020-02-27,采昌國際多媒體股份有限公司,SPO Entertainment Inc.,2,6,0.00,1300.0,28962.0,7037058.0
3906,100,101,美國,音速小子,2020-02-21,美商美國派拉蒙影片股份有限公司台灣分公司,PARAMOUNT PICTURES CORPORATION,1,28,-39.13,3080.0,103984.0,24168050.0
3907,101,102,美國,逃出夢幻島,2020-02-14,美商台灣索尼影業發行股份有限公司台灣分公司,SONY PICTURES RELEASING INTERNATIONAL CORPORAT...,1,83,0.00,9180.0,99406.0,21997167.0
3908,102,103,南韓,紅鞋公主與七矮人,2020-01-30,可樂藝術文創股份有限公司,"FINECUT CO., LTD",2,515,-8.53,26145.0,27906.0,5501812.0


In [4]:
#整體描述
diff = len(df_2020['中文片名'].unique()) - len(df_2020_0['中文片名'].unique()) #268

print('2020至8月底')
print('影片數量:'+str(len(df_2020_0['中文片名'].unique()))+f'(未列入非今年發行之作品{diff}部)')
print('影片國別地區數量:'+str(len(df_2020_0['國別地區'].unique())))
print('發行公司數量:'+str(len(df_2020_0['申請人'].unique())))

2020至8月底
影片數量:416(未列入非今年發行之作品268部)
影片國別地區數量:38
發行公司數量:71


In [5]:
#以國別地區groupby
country_list = df_2020_0['國別地區'].unique() #38
country_gp = df_2020_0.groupby('國別地區')

In [6]:
# 建立新dataframe
subject = ['影片數量','累積總票房']
country_df = pd.DataFrame(index = country_list, columns = subject)

In [7]:
#了解影片數量
country_moviecount = []
country_moviecount_number = []

for country in country_list:
    n = country_gp.get_group(country)
    m = country+","+str(len(n['中文片名'].unique()))
    o = len(n['中文片名'].unique())
    country_moviecount.append(m)
    country_moviecount_number.append(o)
#     print(m)
#     print(o)

In [8]:
#將影片數量加入country_df
country_df['影片數量'] = country_moviecount_number
country_df

,影片數量,累積總票房
中華民國,31,NaN
美國,93,NaN
日本,80,NaN
印度,2,NaN
泰國,11,NaN
俄羅斯,7,NaN
南韓,42,NaN
英國,34,NaN
西班牙,6,NaN
法國,39,NaN


In [11]:
#了解累積票房金額
country_list = df_2020_0['國別地區'].unique()

all_boxoffice = []
country_boxoffice = []
country_boxoffice_number = []

for country in country_list:
    a = country_gp.get_group(country)
    name_list = a['中文片名'].unique()

    name_boxoffice = []

    for name in name_list:
        b = a.groupby('中文片名')
        c = b.get_group(name)
        d = name + ',' + str(max(c['累計銷售金額']))
        e = max(c['累計銷售金額'])
        name_boxoffice.append(e)
        #print(d)
        #print(e)

    f = country + ',' + str(sum(name_boxoffice))
    g = sum(name_boxoffice)
    country_boxoffice.append(f)
    country_boxoffice_number.append(g)
    
    h = sum(country_boxoffice_number)
    all_boxoffice.append(g)
#         print(f)
#         print(g)

In [12]:
#將累積票房加入country_df
country_df['累積總票房'] = country_boxoffice_number
country_df

,影片數量,累積總票房
中華民國,31,2.641226e+08
美國,93,1.097087e+09
日本,80,2.568640e+08
印度,2,1.424921e+06
泰國,11,2.801960e+07
俄羅斯,7,1.194773e+07
南韓,42,5.143503e+08
英國,34,2.256689e+07
西班牙,6,5.882530e+06
法國,39,1.587940e+07


In [13]:
#存成csv
Result ='C:/Users/ching/Desktop/practice/movie/data/df_only2020_bycountry.csv'
country_df.to_csv(Result, index=True )
print( '成功產出'+ Result)

成功產出C:/Users/ching/Desktop/practice/movie/data/df_only2020_bycountry.csv


In [ ]:
#以影片數量排序
country_df.sort_values(by='影片數量', ascending=False, axis=0 )

In [ ]:
#以累積總票房排序
country_df.sort_values(by='累積總票房', ascending=False, axis=0 )